### Data Processing Scripts

Libraries

In [ ]:
import re
import pandas as pd
import pickle
import psycopg2
import psycopg2.extras
import os

Batch Control

In [ ]:
bcfcsv = 'codebook_batches_15.csv'
bcf = pd.read_csv(r'D:\\Documents\\conversion\\1_data import\\'+bcfcsv)
batchproc = bcf['batch'].tolist()
dctp = bcf.loc[bcf['batch'].isin(batchproc)].to_dict(orient='records')


Codebook Functions

In [ ]:
def read_codebook(cbkpickle):
    cbdf = pd.read_pickle(cbkpickle)
    return cbdf

def create_column_list(cbdf):
    x = cbdf['length'].astype('str')
    cbdf['column_name'] = cbdf['variable'].str.lower() + ' char('+x+')'
    column_list = cbdf['column_name'].tolist()
    print (len(column_list),'columns')
    return column_list

def create_slice_list(cbdf):
    slice_list = cbdf['slice_obj'].tolist()
    return slice_list

Data Processing Functions

In [ ]:
def slice_record(line, slices):
    limit = len(line)
    rlist = []
    for i in slices:
        len_slice = len(line[i])
        begin = limit
        limit = limit - len_slice
        if limit <= 0: break
        rlist.append(line[i])
    textrec = '|'.join(rlist)+'\n'
    return textrec

def read_datafile(datafile, slice_list):
    with open(datafile,'r') as f:
        records = []
        print('Processing Records from '+datafile)
        while (True):
            line = f.readline()
            if line == "": break
            records.append((slice_record(line, slice_list)))
    print ('Done')
    print(len(records),'records')
    return records

def write_datafile(records, outfile):
    with open (outfile,'w') as wf:
        for i in records:
            wf.write(i)

In [ ]:
for i in dctp:
    print (i['path'])

### Processing Data into Datafile

In [ ]:
for i in dctp:
    cbkfile = i['path']+'\\'+i['cbkname']+'.pickle'
    datafile = i['path']+'\\'+i['datafile']
    outfile = i['path']+'\\'+i['cendata']+'.text'
    cbdf = read_codebook(cbkfile)
    slice_list = create_slice_list(cbdf)
    data = read_datafile(datafile,slice_list)
    write_datafile(data,outfile)

Database Functions

In [ ]:
def open_database(database):
    conn = psycopg2.connect(dbname=database, user='postgres', password = '3701', host='localhost', port='5432')
    return conn

def create_cursor():
    cursor = conn.cursor()
    return cursor

def set_census_schema():
    cursor.execute("SET SCHEMA 'production'")

def close_database():
    #insert rollback logic
    print('committing the records')
    conn.commit()
    conn.close() 

SQL Functions

In [ ]:
def create_table(tablename, columns):
    columns_text = '(' + ', '.join(columns) + ');'
    cursor.execute('DROP TABLE IF EXISTS ' + tablename + ';')
    cursor.execute('CREATE TABLE ' + tablename + ' ' + columns_text+';')
    conn.commit()

def insert_values(tablename, column_list):
    #slice is one record
    columns_text = '(' + ', '.join(column_list) + ');'
    cursor.execute('INSERT CREATE TABLE ' + tablename + ' ' + columns_text+';')

def clear_table(tablename):
    clear = 'DELETE FROM ' + tablename

def insert_data(datafile, slice_list):
    with open(datafile,'r') as f:
        records = []
        print('Processing Records '+'Batch '+str(batch))
        while (True):
            line = f.readline()
            if line == "": break
            records.append((slice_record(line, slice_list)))
    print ('Done')
    print(len(records),'records')
    return records